## Limpieza de datos y separación del texto por oraciones
## Después se generan los embeddings para cada noticia

## Limpieza de datos y separación por oraciones

Se carga el conjunto de datos (noticia original y el resumen de la noticia)

In [1]:
import nltk
import nltk.data
nltk.download('stopwords')
nltk.download("book")
from nltk.corpus import stopwords #stopwords en Inglés
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
import os
import re
import glob
import tensorflow as tf
from tqdm import trange
import tensorflow_datasets as tfds
from tqdm import tqdm
import numpy as np
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Nata\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\Nata\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     C:\Users\Nata\AppData\Roaming\nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Nata\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     C:\Users\Nata\AppData\Roaming\nltk_data...
[

In [4]:
#Para los conjuntos de datos en .json
import jsonlines
import json
from summarizer import Summarizer

class obj:     
    def __init__(self, dict1):
        self.__dict__.update(dict1)
  
def dict2obj(dict1):
     
    return json.loads(json.dumps(dict1), object_hook=obj)

lista = []
model = Summarizer()    
with jsonlines.open("/////.jsonl") as reader:
    for dictionary in tqdm(reader):
        for i in dict2obj(dictionary).articles:
            new = i.text
            result = model(new)
            myjson = {
                'resumen': result,
                'noticia': new
            }
            
            lista.append(myjson)

jsonNews = json.dumps(lista)          
with open('/////.json', "w") as file:
    file.write(jsonNews)
    

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
992it [54:15:56, 196.93s/it]


In [2]:
# Conjunto de datos de BBC News 
news_files = sorted(glob.glob("D:/data/txt/BBC_txt/BBCNews_docs/*.txt"))
news_files_summ = sorted(glob.glob("D:/data/txt/BBC_txt/BBCNews_summ/*.txt"))

In [2]:
# Conjunto de datos de NY times (Carpeta Your Money) 
news_files = sorted(glob.glob("D:/data/txt/N24_txt/YourM/N24YMNews_docs/*.txt"))
news_files_summ = sorted(glob.glob("D:data/txt/N24_txt/YourM/N24YMNews_summ/*.txt"))

In [2]:
# Conjunto de datos de NY times (Carpeta Global Business) 
news_files = sorted(glob.glob("D:/data/txt/N24_txt/GlobalB/N24GBNews_docs/*.txt"))
news_files_summ = sorted(glob.glob("D:data/txt/N24_txt/GlobalB/N24GBNews_summ/*.txt"))

In [2]:
# Conjunto de datos de NY times (Carpeta Economy)
news_files = sorted(glob.glob("D:/data/txt/N24_txt/Economy/N24ENews_docs/*.txt"))
news_files_summ = sorted(glob.glob("D:data/txt/N24_txt/Economy/N24ENews_summ/*.txt"))

In [2]:
# Conjunto de datos de US-Economic-News 
news_files = sorted(glob.glob("D:/data/txt/USN_txt/USNews_docs/*.txt"))
news_files_summ = sorted(glob.glob("D:data/txt/USN_txt/USNews_summ/*.txt"))

In [3]:
# Conjunto de todas las noticias y de todos los resúmenes
files = np.array(news_files)
files_summ = np.array(news_files_summ)

Limpieza de datos y separación por frases

In [4]:
def clear_spaces(lines): #función que limpia las lineas en blanco y los saltos de linea. Deja todo en una lista de lineas
  ans = lines.copy()
  ans = [line.strip() for line in ans]
  ans = [line for line in ans if line != "" and len(line) > 0 and line !="\n"]
  return ans

In [5]:
def solve_mistakes(lines): #función que pone un espacio después de un punto si no lo tiene. Ejm: markets.And = markets. And
  ans = lines.copy()
  for i in range(len(ans)):
    ans[i] = re.sub(r'"', '', ans[i])
    words = ans[i].split(" ")
    for j in range(len(words)):
      word = words[j].strip()
      k = 1
      is_currency = False
      is_interrog = False
      is_qmarks = False
      while(k+1 < len(word) and not is_currency):
        # print("word[k-1]: ", word[k+1])
        # print("word[k]: ", word[k-1])
        # print("word[k+1]: ", word[k])
        if( (word[k-1].islower() and word[k] == '.' and word[k+1].isupper() ) or (word[k-1].islower() and word[k] == '.' and word[k+1].islower()) or (word[k-1].isupper() and word[k] == '.' and word[k+1].islower()) or (word[k-1].isupper() and word[k] == '.' and word[k+1].isupper()) or (word[k-1].isdigit() and word[k] == '.' and word[k+1].isupper()) or (word[k-1].isdigit() and word[k] == '.' and word[k+1].islower()) or (word[k-1].islower() and word[k] == '.' and word[k+1].isdigit()) or (word[k-1].isupper() and word[k] == '.' and word[k+1].isdigit()) or (word[k-1] == '"' and word[k] == '.' and word[k+1].isupper()) or (word[k-1] == '"' and word[k] == '.' and word[k+1].islower()) or (word[k-1] == '"' and word[k] == '.' and word[k+1] == '"') or (word[k-1].isdigit() and word[k] == '.' and word[k+1] == '"') or (word[k-1].isupper() and word[k] == '.' and word[k+1] == '"') or word[k-1] == "'" and word[k] == '.' and word[k+1].isupper() or word[k-1] == "'" and word[k] == '.' and word[k+1].islower() or word[k-1].islower() and word[k] == '.' and word[k+1] == "'" or word[k-1].isupper() and word[k] == '.' and word[k+1] == "'" or word[k-1] == "'" and word[k] == '.' and word[k+1].isdigit() or word[k-1] == "'" and word[k] == '.' and word[k+1] == "'" or word[k-1].islower() and word[k] == '.' and word[k+1] == "\x89" or word[k-1].isupper() and word[k] == '.' and word[k+1] == "\x89" or word[k-1].isdigit() and word[k] == '.' and word[k+1] == "\x89" or word[k-1] == ')' and word[k] == '.' and word[k+1].isupper() or word[k-1] == ')' and word[k] == '.' and word[k+1].islower() or word[k-1].islower() and word[k] == '.' and word[k+1] == '(' or word[k-1].isupper() and word[k] == '.' and word[k+1] == '(' or word[k-1] == ')' and word[k] == '.' and word[k+1].isdigit() or word[k-1].isdigit() and word[k] == '.' and word[k+1] == '‰Û' or word[k-1].islower() and word[k] == '.' and word[k+1] == '‰Û' or word[k-1].isupper() and word[k] == '.' and word[k+1] == '‰Û' or word[k-1] == '}' and word[k] == '.' and word[k+1].isupper() or word[k-1] == '}' and word[k] == '.' and word[k+1].islower() or word[k-1] == '}' and word[k] == '.' and word[k+1].isdigit() or word[k-1] == ']' and word[k] == '.' and word[k+1].isupper() or word[k-1] == ']' and word[k] == '.' and word[k+1].islower() or word[k-1] == ']' and word[k] == '.' and word[k+1].isdigit() or word[k-1] == '%' and word[k] == '.' and word[k+1].isdigit() or word[k-1] == '%' and word[k] == '.' and word[k+1].islower() or word[k-1] == '%' and word[k] == '.' and word[k+1].isupper() ):
          word = word[:k] + word[k].replace(".",". ",1) + word[k+1:]
        if( (word[k-1].isalpha() or word[k-1].isnumeric()) and (word[k].isalpha() or word[k].isnumeric()) and word[k+1] == '?' ):
          word = word[:k] + word[k] + word[k+1:].replace("?","?. ",1)
        if( word[k-1] == '.' and word[k] == '"' and word[k+1].isupper() ):
          word = word[:k] + word[k].replace('"','" ',1) + word[k+1:]
        if( (word[k+1].isalpha() or word[k-1].isnumeric()) and (word[k].isalpha() or word[k].isnumeric()) and word[k-1] == '%' ):
          word = word[:k] + word[k].replace('"','" ',1) + word[k+1:]
        k += 1
      
      words[j] = str(word)
    ans[i] = " ".join(words)
  return ans

In [6]:
def replace_special_characters_in_words(text_list):
    cleaned_text_list = []
    for text in text_list:
        words = text.split()  
        cleaned_words = []
        for word in words:
            cleaned_word = word.replace('\x89ÛÓ', ' ').replace('\n\n', '.').replace('\n', '.').replace('\x89Û¢', '').replace('\x89Ûª', '').replace('\'', '').replace('\x89ÛÏ', '').replace('\x89Û\x9d', '').replace('\x89ÐÊ', '').replace('\\\\', '').replace('\\', '').replace('\x89Û÷', '').replace('\x89Û_', '').replace('\x89ÏÒ', '')  # Encadenar los reemplazos en una sola línea
            
            cleaned_words.append(cleaned_word)
        cleaned_text = ' '.join(cleaned_words)
        cleaned_text_list.append(cleaned_text)
    return cleaned_text_list

In [8]:
#https://docs.python.org/es/3/howto/regex.html

def split_sentences(text): # función que separa el texto por oraciones (cuando hay un punto), teniendo en cuenta expresiones regulares para evitar separar donde no debe
    sentence_enders = re.compile(r"""
        # Split sentences on whitespace between them.
        (?:               # Group for two positive lookbehinds.
          (?<=[.])      # Either an end of sentence punct,
        | (?<=[.]['"])  # or end of sentence punct and quote.
        )                 # End group of two positive lookbehinds.
        (?<!  \bMr\.   )    # Don't end sentence on "Mr."
        (?<!  \bMrs\.  )    # Don't end sentence on "Mrs."
        (?<!  \bMs\.  )    # Don't end sentence on "Mrs."
        (?<!  \bGov\.  )    # Don't end sentence on "Gov."
        (?<!  \bJr\.   )    # Don't end sentence on "Jr."
        (?<!  \bDr\.   )    # Don't end sentence on "Dr."
        (?<!  \bProf\. )    # Don't end sentence on "Prof."
        (?<!  \bSr\.   )    # Don't end sentence on "Sr."
        (?<!  \bU\.S\.   )   # Don't end sentence on "U.S."
        (?<!  \bD\.C\.   )   # Don't end sentence on "D.C."
        (?<!  \bco\.   )    # Don't end sentence on "co."
        (?<!  \bCorp\.   )    # Don't end sentence on "co."
        (?<!  \bSen\.   )    # Don't end sentence on "co."
        (?<!  \btenn\.   )  # Don't end sentence on "tenn."
        (?<!  \bJan\.   )  # Don't end sentence on "Jan."
        (?<!  \bFeb\.   )  # Don't end sentence on "Feb."
        (?<!  \bMar\.   )  # Don't end sentence on "Mar."
        (?<!  \bApr\.   )  # Don't end sentence on "Apr."
        (?<!  \bMay\.   )  # Don't end sentence on "May."
        (?<!  \bJun\.   )  # Don't end sentence on "Jun."
        (?<!  \bJul\.   )  # Don't end sentence on "Jul."
        (?<!  \bAug\.   )  # Don't end sentence on "Aug."
        (?<!  \bSep\.   )  # Don't end sentence on "Sep."
        (?<!  \bSept\.   )  # Don't end sentence on "Sep."
        (?<!  \bOct\.   )  # Don't end sentence on "Oct."
        (?<!  \bNov\.   )  # Don't end sentence on "Nov."
        (?<!  \bDec\.   )  # Don't end sentence on "Dec."
        (?<!  \bWSJ\.   )  # Don't end sentence on "WSJ."
        (?<!  \bvs\.   )  # Don't end sentence on "vs. "
        (?<!  \bVS\.   )  # Don't end sentence on "VS. "
        (?<!  \bNo\.   )  # Don't end sentence on "No. "
        (?<!  \b[A-Z]\.   )  # Don't end sentence on a capital letter.
        (?<!  \ba\.|\bb\.|\bc\.|\bd\.|\be\.|\bf\.|\bg\.|\bh\.|\bi\.|\bj\.|\bk\.|\bl\.|\bm\.|\bn\.|\bo\.|\bp\.|\bq\.|\br\.|\bu\.|\bv\.|\bw\.|\bx\.|\by\.|\bz\. )  # Don't end sentence on a letter; s and t are removed (ex. don't.)
        (?<!  \b\s1\.\b|\b\s2\.\b|\b\s3\.\b|\b\s4\.\b|\b\s5\.\b|\b\s6\.\b|\b\s7\.\b|\b\s8\.\b|\b\s9\.\b ) # Don't end sentence on a number of one digit.
        (?<!  \b\s10\.\b|\b\s11\.\b|\b\s12\.\b|\b\s13\.\b|\b\s14\.\b|\b\s15\.\b|\b\s16\.\b|\b\s17\.\b|\b\s18\.\b|\b\s19\.\b|\b\s20\.\b|\b\s21\.\b|\b\s22\.\b|\b\s23\.\b|\b\s25\.\b|\b\s26\.\b|\b\s27\.\b|\b\s28\.\b|\b\s29\.\b|\b\s30\.\b|\b\s31\.\b )  # Don't end sentence on a number of two digit.


        \s+               # Split on whitespace between sentences.
        """,
        re.VERBOSE)
    lines_sw = ' '.join(text)
    lines = sentence_enders.split(lines_sw)
    return lines

# Ejemplo
#text = "While working, she began taking wine courses at Napa Valley College, and in 2012, she began working part time at Raymond Vineyards in St. Helena, Calif., as a tasting room associate."
#sentences = split_sentences(text)
#print(sentences)

In [9]:
def stop_words(document_sentences): # función que quita las stopwords
  en_stops = stopwords.words('english')
  i = 0
  for sentence in document_sentences:
    line = sentence.strip().split()

    news_tokens_without_sw = [word for word in line if not word.lower() in en_stops]
    document_sentences[i] = " ".join(news_tokens_without_sw)
    i += 1

  return document_sentences

In [10]:
def separate_words(text): # función que separa el texto por palabras (esto para encontrar la oración más larga de la noticia)
    pattern = r'\b\w+\b|\$[\d,.]+|\€[\d,.]+|\£[\d,.]+|\¥[\d,.]+'
    words = re.findall(pattern, text)
    words = [words[i-1] + words[i] if i > 0 and words[i].isdigit() and words[i-1].endswith(',') else words[i] for i in range(len(words))]
    return words

In [11]:
def make_labels(document_path,summary_path): # funcion que crea el label que representa cuales oraciones del resumen están dentro de la noticia, junto con la noticia separada por oraciones y por palabras
  sentence_labels = []
  general_list = []
  general_list_pw = []
  max_length = 0
  news_text,summary_text = [],[]
  for index, path in enumerate(tqdm(document_path)):
    all_document_sentences = []
    all_document_sentences_pw = []
    with open(path, encoding='utf-8') as file:
      news_text = file.readlines()
      news_text = clear_spaces(news_text)
      news_text = solve_mistakes(news_text)
      news_text = replace_special_characters_in_words(news_text)
      #news_text = stop_words(news_text)
      news_text = split_sentences(news_text)
      news_text = [sentence for sentence in news_text if len(sentence) > 1]
        
    with open(summary_path[index], encoding='utf-8') as file:
      summary_text = file.readlines()
      summary_text = clear_spaces(summary_text)
      summary_text = solve_mistakes(summary_text)
      summary_text = replace_special_characters_in_words(summary_text)
      #summary_text = stop_words(summary_text)
      summary_text = split_sentences(summary_text)
      summary_text = [sentence for sentence in summary_text if len(sentence) > 1]
      

    for id, doc_sent in enumerate(news_text):
      for sum_sent in summary_text:
        if sum_sent in doc_sent or doc_sent in sum_sent:
          sl = 1
          break
        else:
          sl = 0
      sentence_labels.append(sl)
      vect = doc_sent.lower()
      all_document_sentences.append(vect)
      all_document_sentences_pw.append(separate_words(vect))
    general_list.append(all_document_sentences)
    general_list_pw.append(all_document_sentences_pw)


  return general_list, tf.keras.utils.to_categorical(np.array(sentence_labels)), general_list_pw

In [12]:
sentences, labels, pw = make_labels(files, files_summ)

100%|██████████| 7332/7332 [00:30<00:00, 243.63it/s]


In [13]:
np.asarray(sentences).shape

C:\Users\Nata\anaconda3\envs\pruebaRNN\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(7332,)

In [14]:
def create_general_dataset(doc_list): # función que convierte la lista de documentos divididos en oraciones, en una sola lista con todas las oraciones juntas, esto es solo para entrenar los embeddings
  sentences = []
  for docs in doc_list:
    for sent in docs:
      sentences.append(sent)
  return sentences

In [15]:
#ejecutar
general_doc = create_general_dataset(sentences)

In [16]:
# Para calcular la oracion más larga de cada grupo de noticias (número que irá en el output_sequence_length)
len_sent = []
len_sent_id = []
for idoc, doc in enumerate(pw):
  for isent, sent in enumerate(doc):

    len_sent.append(len(sent))
    len_sent_id.append([idoc, isent, len(sent)])

max_len = np.argmax(np.array(len_sent))
for i in range(len(len_sent_id)):
  if len_sent[max_len] == len_sent_id[i][2]:
    print("Sentence max length: ", len_sent[max_len])
    print("Sentence max length text: ", sentences[len_sent_id[i][0]][len_sent_id[i][1]])
    print("Sentence max length ubi: ", news_files[len_sent_id[i][0]])


Sentence max length:  123
Sentence max length text:  yet there is no mention of such very telling numbers as the increase in german marriages from about 511,000 in 1932 to 611,000 in 1936; the jump from 921,000 births in 1932 to 1,280,000 in 1936, meaning that for every two children born in germany in 1932, three were born just four years later; the fact that in 1938 and 1939, most marriages in all of europe were registered in germany, exceeding the numbers among even the prolific people of eastern europe; or the statistic that suicides committed by young people under 20 dropped by 80 percent (!) during the first six years of the hitler regime -- all symptoms of a great and ominous rise of german national confidence.
Sentence max length ubi:  D:/data/txt/USN_txt/USNews_docs\830985624.txt


## GloVe

### Vocabulario

In [17]:
sentences_arr = np.asarray(general_doc[:54])

In [18]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vectorizer = TextVectorization(max_tokens=None, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(sentences_arr).batch(128)
vectorizer.adapt(text_ds)

In [20]:
# Se obtienen los indices de cada palabra
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [49]:
print(voc)

['', '[UNK]', 'i', 'am', 'wheels', 'wally', 'hello', 'have', 'computer', 'a']


In [ ]:
print(word_index)

{'': 0, '[UNK]': 1, 'said': 2, 'us': 3, 'year': 4, 'mr': 5, 'would': 6, 'also': 7, 'market': 8, 'new': 9, 'growth': 10, 'company': 11, 'last': 12, 'economy': 13, 'firm': 14, 'sales': 15, 'bank': 16, 'could': 17, 'economic': 18, '2004': 19, 'government': 20, 'oil': 21, 'years': 22, 'shares': 23, 'however': 24, 'may': 25, 'prices': 26, 'one': 27, 'chief': 28, 'firms': 29, 'two': 30, 'business': 31, 'companies': 32, 'the': 33, 'analysts': 34, 'deal': 35, 'rise': 36, 'expected': 37, 'financial': 38, 'yukos': 39, '2005': 40, 'since': 41, 'group': 42, 'uk': 43, 'months': 44, 'stock': 45, 'december': 46, 'first': 47, 'dollar': 48, 'three': 49, 'still': 50, 'president': 51, 'many': 52, 'time': 53, 'rate': 54, 'european': 55, 'trade': 56, 'people': 57, 'china': 58, 'told': 59, 'investment': 60, 'state': 61, 'interest': 62, 'demand': 63, 'figures': 64, '2003': 65, 'rates': 66, 'profits': 67, 'quarter': 68, 'made': 69, 'spending': 70, 'countries': 71, 'executive': 72, 'news': 73, 'world': 74, 'bi

### GloVe

In [20]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


^C


In [21]:
#path_to_glove_file = "glove.6B.200d.txt"
path_to_glove_file = "glove.6B/glove.6B.200d.txt"

embeddings_index = {}
with open(path_to_glove_file, encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
print(word,coefs)

sandberger [-5.1113e-01 -4.7518e-01  2.2871e-01  8.6524e-03 -4.3737e-01 -2.8747e-01
  2.3416e-01 -2.0332e-02  5.0697e-01 -2.4367e-01 -2.8646e-01 -2.4133e-02
 -9.6845e-06  4.8092e-02 -2.4467e-01 -1.2121e-01  1.3644e-01 -1.6190e-01
  9.9349e-02  3.6545e-02 -3.1657e-02 -8.4172e-01  2.3022e-01  3.0332e-02
 -6.7638e-01 -2.9399e-01  1.3298e-01 -3.7917e-02 -1.0970e-01  2.6541e-03
 -4.6669e-01  1.2329e-01 -4.0373e-03 -2.8782e-01 -1.6733e-01  2.0938e-01
  4.9163e-01 -6.0818e-02  1.4326e-01 -1.3628e-01 -1.5650e-01 -8.3060e-02
 -8.5820e-02 -1.9864e-01  3.6016e-01 -1.6752e-01 -6.4389e-03  2.3173e-01
 -1.6636e-01 -1.7120e-01  1.8400e-01 -5.2889e-01 -7.0440e-02 -3.7772e-01
 -6.6473e-02  3.8519e-01  2.6262e-01 -3.9148e-02 -2.8182e-01 -1.6908e-01
 -4.9491e-01 -1.4138e-01  4.1732e-01 -6.8889e-02  1.0125e-01  1.8499e-02
 -6.6256e-02  1.9328e-01 -2.4784e-01  2.5005e-01 -6.3437e-02  2.6013e-01
  2.3620e-01 -2.9019e-01  4.9803e-01 -7.4534e-01  2.7620e-01  1.6096e-01
 -2.4558e-01  1.6969e-01 -1.1854e-01  7.

In [22]:
num_tokens = len(voc) + 2
embedding_dim = 200
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 584 words (25 misses)


In [23]:
from tensorflow.keras.layers import Embedding
from tensorflow import keras

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [24]:
embedding_matrix.shape

(611, 200)

# Crear modelo de conversión

In [25]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorizer)
model.add(embedding_layer)

In [27]:
def save_embeddings(model, document_path,summary_path):

  news_text,summary_text = [],[]
  for index, path in enumerate(document_path):
    sentence_labels = []
    embeddings = []
    with open(path, encoding='utf-8') as file:
      news_text = file.readlines()
      news_text = clear_spaces(news_text)
      news_text = solve_mistakes(news_text)
      news_text = replace_special_characters_in_words(news_text)
      #news_text = stop_words(news_text)
      news_text = split_sentences(news_text)



    with open(summary_path[index], encoding='utf-8') as file:
      summary_text = file.readlines()
      summary_text = clear_spaces(summary_text)
      summary_text = solve_mistakes(summary_text)
      summary_text = replace_special_characters_in_words(summary_text)
      #summary_text = stop_words(summary_text)
      summary_text = split_sentences(summary_text)


    for id, doc_sent in enumerate(news_text):
      for sum_sent in summary_text:
          if sum_sent in doc_sent or doc_sent in sum_sent:
            sl = 1
            break
          else:
            sl = 0
      sentence_labels.append(sl)
      embeddings.append(model.predict([doc_sent]))
    filename = path.split("/")[-1].split(".")[0].split("\\")[-1]
    docs_embeddings = np.asarray(embeddings, dtype = np.float32) 
    sent_labels = np.asarray(sentence_labels, dtype = np.int8) 
    np.save("D:/data/Embeddings_npy/USNews/USN_docs/" + filename, docs_embeddings)
    np.save("D:/data/Embeddings_npy/USNews/USN_label/" + filename, sent_labels) 

In [28]:
#Para cargar todos
save_embeddings(model, news_files, news_files_summ)